# 神经网络：感知机Perceptron

## Udacity：
![title](../images/perceptron_u.png)

## 李航：
感知机（perceptron）是二类分类的线性分类模型，其输入为实例的特征向量，输出为实例的类别，取 +1 和 -1 二值。   
感知机对应于输入空间（特征空间）中将实例划分为正负两类的分离超平面，属于判别模型。   
感知机学习旨在求出将训练数据进行线性划分的分离超平面，为此，导入基于误分类的损失函数，利用梯度下降法对损失函数进行极小化，求得感知机模型。   
感知机学习算法具有简单而易于实现的优点，分为原始形式和对偶形式。    
感知机预测是用学习得到的感知机模型对新的输入实例进行分类。  
感知机1957年由Rosenblatt提出，是神经网络与支持向量机的基础。 

### 感知机模型
感知机定义：   
假设输入空间（特征空间）$X \subseteq R^{n}$，输出空间是$y =\{+1, -1\}$。输入$x\in{X}$表示实例的特征向量，对应于输入空间（特征空间）的点；输出$y\in Y$表示实例的类别。   
由输入空间到输出空间的函数：
$$f(x) = sign(w\cdot x +b)$$ 称为感知机。   

其中，$w$和$b$为感知机模型参数，$w\in R^{n}$叫做权值（weight）或权值向量(weight vector)，$b\in R$叫做偏置(bias)，     
$w \cdot x$表示$w$和$x$的内积。    
sign是符号函数，即$$sign(x) = \begin{cases} +1, & x \ge 0 \\ -1, & x < 0 \end{cases}$$


感知机是一种线性分类模型，属于判别模型。   
感知机模型的假设空间是定义在特征空间中的所有线性分类模型（linear classification model）或线性分类器（linear classifier），即函数集合$\{ f | f(x) = w \cdot x + b\}$。


感知机的几何解释：  
线性方程：$$w \cdot x + b = 0$$
对应于特征空间$R^{n}$中的一个超平面$S$，其中$w$是超平面的法向量，$b$是超平面的截距。 这个超平面将特征空间划分为两个部分。   
位于两部分的点（特征向量）分别被分为正、负两类。   
因此，超平面$S$称为分离超平面(separating hyperplane)。

![title](../images/perceptron_model1.png)


感知机学习，由训练数据集（实例的特征向量及类别）  
$$ T = \{(x_1，y_1), (x_2, y_2), ... , (x_N, y_N)\}$$   
其中，$x_i \in X = R^n$，$y_i \in Y = \{+1, -1\}$，$i = 1,2,...,N$，求得感知机模型，即求得模型参数$w$，$b$。   
感知机预测，通过学习得到的感知机模型，对于新年的输入实例给出其对应的输出类别。

### 感知机的学习策略（损失函数）
### 感知机学习算法：原始形式、对偶形式
### 算法的收敛性